In [ ]:
import os
import pandas as pd
import tempfile
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.dml.color import RGBColor
from pptx.util import Cm, Pt
import numpy as np
import math



# Dossier (spécifique à Notebook jupyter)
SCRIPT_DIR = os.getcwd()  # répertoire courant du notebook
ROOT_DIR = os.path.normpath(os.path.join(SCRIPT_DIR, ".."))


# Fichiers principaux
EXCEL_FILE = os.path.join(ROOT_DIR, "data", "donnees_solaire.xlsx")
PPTX_FILE = os.path.join(ROOT_DIR, "modele_fiche.pptx")
OUTPUT_FILE = os.path.join(ROOT_DIR, "outputs", "fiches_solaire_remplies.pptx")

# Images
LOGO_DIR = os.path.join(ROOT_DIR, "data", "Logo_image")
CHECK_IMG = os.path.join(LOGO_DIR, "check.png")
CROSS_IMG = os.path.join(LOGO_DIR, "cross.png")
LOGO_REGLEMENTATION = os.path.join(LOGO_DIR, "reglementation.png")
LOGO_TOPSITE = os.path.join(LOGO_DIR, "top_site.png")
QGIS_IMG_DIR = os.path.join(ROOT_DIR, "data", "QGIS_image")

# Couleurs
REG_FILL_COLOR = RGBColor(0x45, 0x1D, 0xC7)  # Bleu réglementaire
PV_FILL_COLOR = RGBColor(0x04, 0xF0, 0x6A)   # Vert PV


# ====================================================
# 🧾 LECTURE DES FICHIERS
# ====================================================

df = pd.read_excel(EXCEL_FILE, dtype=str, header=1)
prs = Presentation(PPTX_FILE)

# ====================================================
# 🧮 CALCUL DES SITES ET DES FICHIERS
# ====================================================
MAX_SLIDES_PER_FILE = 200  # Nombre maximal de sites par fichier
total_sites = len(df)
num_files = math.ceil(total_sites / MAX_SLIDES_PER_FILE)

# ====================================================
# 🖼️ INSERTION D’IMAGE DE SITE
# ====================================================
def insert_site_image(slide, row, images_folder=QGIS_IMG_DIR, width=Cm(11.5)):
    site_name = str(row.get("Code_site", "")).strip()
    if not site_name:
        return

    possible_ext = [".png", ".jpg", ".jpeg", ".bmp"]
    image_path = None
    for ext in possible_ext:
        test_path = os.path.join(images_folder, f"{site_name}{ext}")
        if os.path.exists(test_path):
            image_path = test_path
            break

    if not image_path:
        print(f"⚠️ Aucune image trouvée pour {site_name}")
        return

    left = Cm(1.5)
    top = Cm(3.5)
    slide.shapes.add_picture(image_path, left, top, width=width)
    print(f"✅ Image ajoutée pour {site_name}")


# ====================================================
# 🧩 REMPLACEMENT DES PLACEHOLDERS
# ====================================================
def replace_text_in_textframe(text_frame, row):
    for paragraph in text_frame.paragraphs:
        text = "".join(run.text for run in paragraph.runs)
        for col in row.keys():
            placeholder = f"{{{{{col}}}}}"  # {{NomColonne}}
            if placeholder in text:
                text = text.replace(placeholder, str(row[col]))
        for run in paragraph.runs:
            run.text = ""
        if paragraph.runs:
            paragraph.runs[0].text = text
        else:
            paragraph.add_run().text = text



def replace_text_in_shape(shape, row):
    if hasattr(shape, "shapes") and len(shape.shapes) > 0:
        for subshape in shape.shapes:
            replace_text_in_shape(subshape, row)
        return
    if hasattr(shape, "table"):
        for r in shape.table.rows:
            for c in r.cells:
                if c.text_frame:
                    replace_text_in_textframe(c.text_frame, row)
        return
    if shape.has_text_frame:
        replace_text_in_textframe(shape.text_frame, row)


# ====================================================
# 🧱 COLORATION DU TABLEAU RÉGLEMENTATION
# ====================================================
def fill_table_cells_by_excel(slide, row):
    for shape in slide.shapes:
        if hasattr(shape, "table"):
            table = shape.table
            for r in table.rows:
                cell_text = r.cells[0].text.strip()
                if cell_text in row and str(row[cell_text]).upper() == "X":
                    r.cells[0].fill.solid()
                    r.cells[0].fill.fore_color.rgb = REG_FILL_COLOR


# ====================================================
# 🌞 COLORATION TABLEAU PV
# ====================================================
def fill_pv_table_by_excel(slide, row):
    for shape in slide.shapes:
        if hasattr(shape, "table"):
            table = shape.table
            for r in table.rows:
                for cell in r.cells:
                    cell_text = cell.text.strip()
                    if cell_text in ["PV Toîture", "PV Sol", "Ombrière"]:
                        if str(row.get(cell_text, "")).upper() == "X":
                            cell.fill.solid()
                            cell.fill.fore_color.rgb = PV_FILL_COLOR


# ====================================================
# ✅ AJOUT DE LOGOS (CHECK / CROSS)
# ====================================================
def insert_logo_in_cell(slide, table_shape, row_idx, col_idx, img_path, width=Cm(0.4), height=Cm(0.4)):
    table = table_shape.table
    left = table_shape.left + sum(table.columns[i].width for i in range(col_idx)) + (table.columns[col_idx].width - width) / 2
    top = table_shape.top + sum(table.rows[i].height for i in range(row_idx)) + (table.rows[row_idx].height - height) / 2
    slide.shapes.add_picture(img_path, left, top, width=width, height=height)


def fill_type_conso_table(slide, row):
    """
    Insère CHECK ou CROSS dans la colonne juste après "Électrique" et "Gaz Nat."
    selon les valeurs Excel.
    """
    # Texte complet de la ligne Excel en minuscule
    row_texts = " ".join(str(v).lower() for v in row.values)
    has_elec = "électrique" in row_texts
    has_gaz = "gaz" in row_texts or "gaz nat" in row_texts

    for shape in slide.shapes:
        if not hasattr(shape, "table"):
            continue

        table = shape.table
        header_row = table.rows[0]

        elec_col_idx = None
        gaz_col_idx = None
        # Repère les colonnes dans l'entête
        for c_idx, cell in enumerate(header_row.cells):
            text = cell.text.strip().lower()
            if "électrique" in text:
                elec_col_idx = c_idx
            if "gaz nat" in text or text == "gaz":
                gaz_col_idx = c_idx

        # Insert logos dans la colonne juste après
        for r_idx, row_obj in enumerate(table.rows):
            if elec_col_idx is not None and elec_col_idx + 1 < len(row_obj.cells):
                img = CHECK_IMG if has_elec else CROSS_IMG
                insert_logo_in_cell(slide, shape, r_idx, elec_col_idx + 1, img)

            if gaz_col_idx is not None and gaz_col_idx + 1 < len(row_obj.cells):
                img = CHECK_IMG if has_gaz else CROSS_IMG
                insert_logo_in_cell(slide, shape, r_idx, gaz_col_idx + 1, img)


# ====================================================
# 🏷️ LOGO SÉLECTION TYPE
# ====================================================
def fill_selection_type_table(slide, row):
    selection_value = str(row.get("Sélection type", "")).strip().lower()

    for shape in slide.shapes:
        if not hasattr(shape, "table"):
            continue
        table = shape.table
        first_cell_text = table.cell(0, 0).text.strip().lower()

        if "sélection" in first_cell_text:
            if "réglementaire" in selection_value:
                img_path = LOGO_REGLEMENTATION
            elif "top site" in selection_value:
                img_path = LOGO_TOPSITE
            else:
                continue
            insert_logo_in_cell(slide, shape, 0, 1, img_path, width=Cm(0.8), height=Cm(0.8))

# ====================================================
# CREATION GRAPH CONSO ET PROD
# ====================================================
def create_monthly_bar_chart(row, site_name, width=12, height=6):
    """
    Crée un graphique en barres superposées (consommation vs production) pour un site.
    row : dictionnaire de la ligne Excel
    site_name : nom du site (pour le titre)
    width, height : taille du graphique en pouces
    Retour : chemin vers le fichier image PNG
    """
    # Colonnes pour consommation et production
    months = ["Janvier", "Février", "Mars", "Avril", "Mai", "Juin",
              "Juillet", "Août", "Septembre", "Octobre", "Novembre", "Décembre"]
    months_abbr = ["Jan", "Fev", "Mar", "Avr", "Mai", "Jun",
              "Jul", "Aou", "Sep", "Oct", "Nov", "Dec"]
        
    x = np.arange(len(months))
    width_bar = 0.35  # largeur de chaque barre
    fig, ax = plt.subplots(figsize=(width, height))

    # Extraction des données
    consumption = [float(row.get(f"Conso {m}", 0) or 0) for m in months]
    production  = [float(row.get(f"Prod {m}", 0) or 0) for m in months]
    
    # Consommation en bas
    ax.bar(x - width_bar/2, consumption, width_bar, label="Consommation", color="#FFCA4A", alpha=0.8)
    # Production empilée au-dessus de la consommation
    ax.bar(x + width_bar/2, production, width_bar, label="Production", color="#4682B4", alpha=1)
    # Mise en forme
    ax.set_xticks(x)
    ax.set_xticklabels(months_abbr, ha="center")
    ax.set_ylabel("Consommation actuelle et production potentielle (kWh)")
    ax.legend()
    ax.grid(axis="y", linestyle="--", alpha=0.5)

    # Sauvegarde dans un fichier temporaire
    tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
    plt.savefig(tmp_file.name, bbox_inches="tight")
    plt.close(fig)
    return tmp_file.name


def insert_chart_on_slide(slide, chart_path, left=Cm(14.25), top=Cm(3.9), width=Cm(11.5)):
    """
    Insère un graphique en image sur la slide.
    left, top : position en cm
    width : largeur du graphique, la hauteur est ajustée automatiquement
    """
    slide.shapes.add_picture(chart_path, left, top, width=width)

# ====================================================
# 🚀 BOUCLE PRINCIPALE
# ====================================================

for file_idx in range(num_files):
    # Charge le modèle pour chaque fichier
    prs = Presentation(PPTX_FILE)
    
    start_idx = file_idx * MAX_SLIDES_PER_FILE
    end_idx = min(start_idx + MAX_SLIDES_PER_FILE, total_sites)
    df_chunk = df.iloc[start_idx:end_idx]

    for local_idx, (_, row_data) in enumerate(df_chunk.iterrows()):
        slide = prs.slides[local_idx]  # On utilise la slide déjà présente
        site_name = row_data.get("Code_site", f"Site {start_idx + local_idx + 1}")

        # 1️⃣ Remplacement texte
        for shape in slide.shapes:
            replace_text_in_shape(shape, row_data)

        # 2️⃣ Tableau réglementation
        fill_table_cells_by_excel(slide, row_data)

        # 3️⃣ Type conso (logos)
        fill_type_conso_table(slide, row_data)

        # 4️⃣ Image du site
        insert_site_image(slide, row_data)

        # 5️⃣ Sélection type
        fill_selection_type_table(slide, row_data)

        # 6️⃣ Graphique consommation vs production
        chart_file = create_monthly_bar_chart(row_data, site_name)
        insert_chart_on_slide(slide, chart_file)
        os.remove(chart_file)
        total_sites = len(df)

    # Sauvegarde du fichier partiel
    output_file_part = OUTPUT_FILE.replace(".pptx", f"_{file_idx+1}.pptx")
    prs.save(output_file_part)
    print(f"✅ PowerPoint sauvegardé : {output_file_part}")

